# Exporting HuggingFace Models To ONNX Format #

### Export some embedding models from HF to ONNX ###

In [2]:

from transformers import AutoTokenizer, AutoModel
import torch

c:\Users\User\Documents\SideProject\DJL_Experiments-main\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Exporting mxbai-embed-large-v1 embedding model

model_name = "mixedbread-ai/mxbai-embed-large-v1"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
test_input = "Hello Hazrul"
inputs = tokenizer(test_input, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    print(outputs)


input_text = tokenizer(test_input, return_tensors="np")
print(input_text)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.3848, -0.1683,  0.1436,  ..., -0.1220,  0.3842, -0.3292],
         [ 0.2530,  0.0114,  0.3862,  ..., -0.4523,  0.2744, -0.3398],
         [-0.1343,  0.2003,  0.3286,  ...,  0.0507,  0.9975, -0.5576],
         ...,
         [ 0.3053, -0.1917,  0.4317,  ..., -0.0707,  0.8447, -0.4580],
         [ 0.1495, -0.3793,  0.6162,  ...,  0.2146,  0.7438, -0.1687],
         [-0.0203, -0.5344,  0.5132,  ..., -0.1450,  0.4284, -0.6931]]]), pooler_output=tensor([[-0.9023, -0.4622, -0.7371,  ...,  0.3709,  0.8697, -0.7646]]), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)
{'input_ids': array([[ 101, 7592, 5292, 2480, 6820, 2140,  102]]), 'token_type_ids': array([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1]])}


In [7]:
onnx_export_path = "mxbai-embed-large-v1.onnx"

torch.onnx.export(
    model,
    # Provide a tuple of inputs; adjust if your model requires more (like token_type_ids)
    (inputs["input_ids"], inputs["attention_mask"]),
    "mxbai_embed_large_v1.onnx",  # Output ONNX file path
    input_names=["input_ids", "attention_mask"],  # These names will be used in ONNX
    output_names=["output"],  # Modify based on your model’s actual output(s)
    dynamic_axes={
         "input_ids": {0: "batch_size", 1: "sequence_length"},
         "attention_mask": {0: "batch_size", 1: "sequence_length"},
         "output": {0: "batch_size"}
    },
    opset_version=14,  # or a higher version if needed
)
